# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_path = '../WeatherPy/output_data/cities.csv'
cities_df = pd.read_csv(cities_data_path)
print(cities_df.count())
cities_df.head()

City          559
Cloudiness    559
Country       554
Date          559
Humidity      559
Lat           559
Lng           559
Max Temp      559
Wind Speed    559
dtype: int64


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,New Norfolk,75,AU,2020-04-01,87,-42.78,147.06,60.008,2.236940
1,Busselton,98,AU,2020-04-01,48,-33.65,115.33,66.002,13.533487
2,San Cristobal,75,VE,2020-04-01,88,7.77,-72.22,77.000,2.236940
3,Marzuq,9,YE,2020-04-01,26,14.40,46.47,87.710,13.958506
4,Yinchuan,60,CN,2020-04-01,13,38.47,106.27,41.000,4.004123


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) # Fill in with your API key

locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']

In [4]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50,
                                 point_radius = 2, opacity=0.5))
embed_minimal_html('./output_data/vacation_heatmap.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vac_cities_df = cities_df.copy()
vac_cities_df = vac_cities_df.loc[(vac_cities_df['Max Temp'] > 70) & 
                                  (vac_cities_df['Max Temp'] < 80) & 
                                  (vac_cities_df['Wind Speed'] < 10) &
                                  (vac_cities_df['Cloudiness'] == 0)]

vac_cities_df.count()

City          4
Cloudiness    4
Country       4
Date          4
Humidity      4
Lat           4
Lng           4
Max Temp      4
Wind Speed    4
dtype: int64

In [6]:
vac_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
94,Barra de Tecoanapa,0,MX,2020-04-01,75,16.52,-98.75,74.246,2.751436
213,Acajutla,0,SV,2020-04-01,90,13.59,-89.83,71.006,7.896398
397,Yeppoon,0,AU,2020-04-01,83,-23.13,150.73,73.400,4.697574
527,Bam,0,IR,2020-04-01,14,29.11,58.36,73.256,5.346287


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vac_cities_df.copy()

In [8]:
for index, data in hotel_df.iterrows():
    print(index, data['Lat'], data['Lng'])
    latitude = data['Lat']
    longitude = data['Lng']
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    hotel_name = response['results'][0]['formatted_address']
    hotel_df.loc[index, 'Hotel Name'] = hotel_name
    print(hotel_name)
    

94 16.52 -98.75
Barra de Tecoanapa, Guerrero, Mexico
213 13.59 -89.83
Acajutla, El Salvador
397 -23.13 150.73
Yeppoon QLD 4703, Australia
527 29.11 58.36
Bam, Kerman Province, Iran


In [9]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
94,Barra de Tecoanapa,0,MX,2020-04-01,75,16.52,-98.75,74.246,2.751436,"Barra de Tecoanapa, Guerrero, Mexico"
213,Acajutla,0,SV,2020-04-01,90,13.59,-89.83,71.006,7.896398,"Acajutla, El Salvador"
397,Yeppoon,0,AU,2020-04-01,83,-23.13,150.73,73.400,4.697574,"Yeppoon QLD 4703, Australia"
527,Bam,0,IR,2020-04-01,14,29.11,58.36,73.256,5.346287,"Bam, Kerman Province, Iran"


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50, point_radius = 2, opacity=1))
fig.add_layer(gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info))
embed_minimal_html('./output_data/vacation_hotels.html', views=[fig])
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))